  Author: Ankit Kariryaa, University of Bremen
  
  Modified by Xuehui Pi and Qiuqi Luo



In [1]:
from tensorflow.keras.models import load_model
import numpy as np               # numerical array manipulation
import pandas as pd
import geopandas as gps
import os
import time
from collections import defaultdict
from functools import reduce
from PIL import Image
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.warp             # Reproject raster samples
from shapely.geometry import Point, Polygon
from shapely.geometry import mapping, shape
import fiona
import cv2
from tqdm import tqdm
import rasterio.mask
import affine

from core.UNet import UNet
from core.losses import tversky, accuracy, dice_loss, IoU, recall, precision,F1_score
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.frame_info import FrameInfo
from core.dataset_generator import DataGenerator
from core.visualize import display_images

%matplotlib inline
import matplotlib.pyplot as plt  # plotting tools
import matplotlib.patches as patches

import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

import tensorflow as tf
print(tf.__version__)

2024-02-28 10:04:07.404796: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


2.4.1


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2024-02-28 10:04:09.369717: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-28 10:04:09.371643: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-02-28 10:04:09.372607: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-02-28 10:04:09.397570: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-02-28 10:04:09.397598: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] 

In [3]:
# Initialize the data related variables used in the notebook 

# For reading the GSW and annotated images generated in the step - 1

base_dir = r'/home/nkd/hbh'
image_type = '.png'
GSW_fn = 'occurrence'
annotation_fn = 'annotation'

# For testing, images are divided into sequential patches 
patch_generation_stratergy = 'random'
patch_size = (512,512,2) ## Height * Width * (Input or Output) channels：[GSW, ANNOTATION]
BATCH_SIZE = 16 # Model is evaluated in batches; See https://keras.io/models/model/

# # When stratergy == sequential
step_size = (512,512)

input_shape = (512,512,1)
input_image_channel = [0]
input_label_channel = [1]

In [4]:
OPTIMIZER_NAME = 'adaDelta'
OPTIMIZER = adaDelta 
OPTIMIZER=tf.train.experimental.enable_mixed_precision_graph_rewrite(OPTIMIZER)

Instructions for updating:
Use tf.keras.mixed_precision. There is a guide at https://www.tensorflow.org/guide/mixed_precision. Alternatively, `tf.compat.v1.mixed_precision.enable_mixed_precision_graph_rewrite` can be used, but this is not recommended for TF2 code.
  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)


2024-02-28 10:04:09.903484: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-02-28 10:04:09.903897: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-02-28 10:04:09.903937: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-02-28 10:04:09.904013: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-02-28 10:04:09.904025: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2024-02-28 10:04:09.904036: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2024

In [10]:
LOSS = tversky
LOSS_NAME = 'tversky'
modelToEvaluate =  r'/home/nkd/hbh/U_Net_GSW/saved_models/UNet/lakes_20210524-1616_AdaDelta_weightmap_tversky_01_512.h5'
# modelToEvaluate =  r'/home/nkd/hbh/U_Net_GSW/saved_models/UNet/lakes_20240227-1502_AdaDelta_tversky_01_512.h5'

In [6]:
#File path for final report 
timestr = time.strftime("%Y%m%d-%H%M")
chf = input_image_channel + input_label_channel
chs = reduce(lambda a,b: a+str(b),   chf, '')
evaluation_report_path = model_path =  os.path.join(base_dir, 'evaluationreport') 
if not os.path.exists(evaluation_report_path):
    os.makedirs(evaluation_report_path)
evaluation_report_filename = os.path.join(evaluation_report_path,'evaluation_per_pixel{}_{}.csv'.format(timestr,chs))
print(evaluation_report_filename) 

/home/nkd/hbh/evaluationreport/evaluation_per_pixel20240228-1004_01.csv


In [7]:
def readFrames(dataType):
    frames=[]
    print(dataType)
    for i in range(1,7):
        path_to_write=r'/home/nkd/hbh/normal/patchesReshape/{}/type{}'.format(dataType,i)
        all_files = os.listdir(path_to_write)
        all_files_GSW = [fn for fn in all_files if fn.startswith(GSW_fn) and fn.endswith(image_type)]#ndwi.png
        print('type{} image number:{}'.format(i,len(all_files_GSW)))
        for j, fn in enumerate(all_files_GSW):
            GSW_img = rasterio.open(os.path.join(path_to_write, fn))
            read_GSW_img = GSW_img.read()
            comb_img = np.transpose(read_GSW_img, axes=(1,2,0)) #Channel at the end  ( , ,1) 
            annotation_im = Image.open(os.path.join(path_to_write, fn.replace(GSW_fn,annotation_fn)))
            annotation = np.array(annotation_im)
            f = FrameInfo(comb_img, annotation)
            frames.append(f)
    return frames

In [8]:
frames=readFrames('test')
test_patches = DataGenerator(input_image_channel,patch_size, frames, input_label_channel, augmenter = None).all_sequential_patches(step_size)
print('test patchs number:',len(test_patches[0]))

test
type1 image number:36
type2 image number:30
type3 image number:42
type4 image number:51
type5 image number:2
type6 image number:1
test patchs number: 1453


In [11]:
#Evaluate model 
def evaluate_model(model_path, evaluation_report_filename):
    print(model_path, evaluation_report_filename)
    model = load_model(model_path, custom_objects={'tversky': tversky, 'dice_loss':dice_loss, 'accuracy':accuracy , 'recall':recall,'precision':precision,'IoU':IoU}, compile=False)

    model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy,recall,precision,IoU])
    
    print('Evaluating model now!')
    ev = model.evaluate(x=test_patches[0], y=test_patches[1],  verbose=1, use_multiprocessing=False)
    report  = dict(zip(model.metrics_names, ev))
    report['model_path'] =  model_path   
    report['test_frame_dir']= base_dir   
    report['total_patch_count']= len(test_patches[0])  
    return report

report = evaluate_model(modelToEvaluate, evaluation_report_filename)

/home/nkd/hbh/U_Net_GSW/saved_models/UNet/lakes_20210524-1616_AdaDelta_weightmap_tversky_01_512.h5 /home/nkd/hbh/evaluationreport/evaluation_per_pixel20240228-1004_01.csv


OSError: SavedModel file does not exist at: /home/nkd/hbh/U_Net_GSW/saved_models/UNet/lakes_20210524-1616_AdaDelta_weightmap_tversky_01_512.h5/{saved_model.pbtxt|saved_model.pb}

In [31]:
print(report)

# tdf = pd.DataFrame(report, index=[0])  
# print(tdf.columns)
# col_beginning = ['model_path','test_frame_dir', 'total_patch_count']

# col_rest = [x for x in tdf.columns.tolist() if x not in col_beginning]
# cols = col_beginning + col_rest
# tdf = tdf[cols]
# tdf.to_csv(evaluation_report_filename)

{'loss': 0.08131864666938782, 'dice_loss': 0.08217889070510864, 'accuracy': 0.9915927052497864, 'recall': 0.9616610407829285, 'precision': 0.905194103717804, 'IoU': 0.8834644556045532, 'model_path': '/home/nkd/hbh/U_Net_GSW/saved_models/UNet/lakes_20240227-1502_AdaDelta_tversky_01_512.h5', 'test_frame_dir': '/home/nkd/hbh', 'total_patch_count': 1453}
